<a href="https://colab.research.google.com/github/Balajivenkatesh05/Email-Spam-detection/blob/main/email_spam_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [138]:
import pandas as pd

In [139]:
df = pd.read_csv('mail_data.csv')

In [140]:
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [141]:
df.tail()

,Category,Message
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...
5571,ham,Rofl. Its true to its name


In [142]:
df.shape

(5572, 2)

In [143]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [144]:
df.isnull().sum()

Category    0
Message     0
dtype: int64

In [145]:
df.duplicated().sum()

415

In [146]:
df = df.drop_duplicates()

In [147]:
df.duplicated().sum()

0

# Check for Balancing

In [148]:
df.Category.value_counts()

ham     4516
spam     641
Name: Category, dtype: int64

In [149]:
print(f"Percentage of spam:", (641/(4516+641)*100))
print(f"Percentage of ham:", (4516/(4516+641)*100))

Percentage of spam: 12.429707194105099
Percentage of ham: 87.5702928058949


# Encoding

In [150]:
df.Category = df.Category.map({"ham":0, "spam":1})
df.head()

<ipython-input-150-3ffea3442ce3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Category = df.Category.map({"ham":0, "spam":1})


,Category,Message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


# Feature and target selection

In [151]:
x = df['Message']
y = df['Category']
x

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                 Will ü b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: Message, Length: 5157, dtype: object

In [152]:
y

0       0
1       0
2       1
3       0
4       0
       ..
5567    1
5568    0
5569    0
5570    0
5571    0
Name: Category, Length: 5157, dtype: int64

# Splitting

In [153]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2,random_state = 3)

# Feature Extraction using 

In [154]:
from sklearn.feature_extraction.text import TfidfVectorizer
feature_extraction = TfidfVectorizer()

x_train_features = feature_extraction.fit_transform(x_train)
x_test_features = feature_extraction.transform(x_test)

# Balancing

In [155]:
from imblearn.combine import SMOTEENN 
smt = SMOTEENN(sampling_strategy='all') 
x_train_smt, y_train_smt = smt.fit_resample(x_train_features, y_train)

In [156]:
x_train_smt.shape

(7228, 7695)

In [157]:
y_train_smt.shape

(7228,)

# Model - Logistic Regression

In [158]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
#Training the Logistic Regression model with training data
model.fit(x_train_smt,y_train_smt)

LogisticRegression()

In [159]:
from sklearn.metrics import accuracy_score
#predicting accuracy on training data

prediction_on_training_data = model.predict(x_train_smt)
accuracy_on_training_data = accuracy_score(y_train_smt, prediction_on_training_data)
print("Accuracy on Training Data: ", accuracy_on_training_data)

Accuracy on Training Data:  0.9930824571112341


In [160]:
#accuracy on test data

prediction_on_test_data = model.predict(x_test_features)
accuracy_on_test_data = accuracy_score(y_test, prediction_on_test_data)
print("Accuracy on Test Data: ", accuracy_on_test_data)

Accuracy on Test Data:  0.9796511627906976


In [161]:
print("Email Prediction")

input_mail = input(" Enter the Body of Email : ")

# convert text to feature vectors
input_data_features = feature_extraction.transform([input_mail])

#making prediction
prediction  = model.predict(input_data_features)

print(prediction)

if(prediction[0]==0):
  print("\n It is a Ham Mail")

else:
  print("\n It is a Spam Mail")

Email Prediction
 Enter the Body of Email : Hi man
[0]

 It is a Ham Mail
